In [5]:
from __future__ import print_function
import os.path
import httplib2
#import apiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from datetime import datetime


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'
SAMPLE_RANGE_NAME = 'managers'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_852918827851-jphlh48nvfk0rairmigc9vm4kbaaeeeb.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
            print('%s, %s, %s' % (row[0],row[1], row[2]))

    #if __name__ == '__main__':
    #    main()
    
main()


manager_id, d_manager, d_club
1e9c5562-0cfc-11ea-abe1-c412f533dba1, manager #8, club #2
43756fa4-57a0-11e9-ab9a-c412f533dba1, manager #11, club #4
543888c4-fbb3-11e9-abdc-c412f533dba1, manager #12, club #3
5dd88f32-6501-11ea-abf3-c412f533dba1, manager #10, club #2
9a87c608-52dc-11ea-abeb-c412f533dba1, manager #9, club #3
abed5496-ca88-11e9-abb8-c412f533dba1, manager #5, club #4
ad52c7a8-a752-11e7-8115-c412f533dba1, manager #7, club #1
b55a233c-5df6-11ea-abf2-c412f533dba1, manager #2, club #4
b832aac3-fcda-11ea-ac18-c412f533dba1, manager #13, club #3
d12fe765-b444-11e9-abb3-c412f533dba1, manager #1, club #1
db06b8bc-bfb2-11e7-8116-c412f533dba1, manager #4, club #2
e463b03f-2d74-11eb-ac1f-c412f533dba1, manager #3, club #1
f44596c6-12cd-11eb-ac1c-c412f533dba1, manager #6, club #2
